In [27]:
import pandas as pd
%matplotlib inline

import matplotlib.pyplot as plt
pd.options.display.max_rows=10
from datetime import datetime
from statsmodels.tsa.arima_model import ARIMA
import xgboost
import graphviz
import matplotlib.dates as md
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
import scipy
from scipy.stats import spearmanr
from pylab import rcParams
import seaborn as sb
from matplotlib.pylab import rcParams
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller, arma_order_select_ic
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import MyAnalysis 
rcParams['figure.figsize'] = 3,3



import numpy as np
df = pd.read_table("grenoble_7.k7",sep = ',',header = 0)

def trace_conf_mat(cm, acc,classes, norm, title,cmap=plt.cm.Blues):
    accuracy=acc
    if norm:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title+'%.2f' % accuracy,
           ylabel='True label',
           xlabel='Predicted label')
    #plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
     #        rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if norm else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

def plot_confusion_matrix_pdr(df, classes, cm,accuracy, normalize=False, title=None,cmap=plt.cm.Blues):
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'
    trace_conf_mat(cm, accuracy,classes, normalize, title,cmap=plt.cm.Blues)
np.set_printoptions(precision=2)


def my_split(df,src,dst,size_of_test,kind):
    ros = RandomOverSampler(random_state=0)
    rus = RandomUnderSampler(random_state=0)
    target=list()
    target_names=list()
    
    if kind=="all":                                              #it is for pdr+rssi
        combined_features=list()
        for i in range(len(df)):
            combined_features.append(df['pdr'][i]*df['pdr'][i]+df['mean_rssi'][i]*df['mean_rssi'][i])
            if df['pdr'][i] < 0.3:
                target.append(0)
                target_names.append('Bad')
            elif df['pdr'][i]>0.75:
                target.append(2)
                target_names.append('Good')
            elif ((df['pdr'][i]>=0.3 and df['pdr'][i]<=0.75)and df['mean_rssi'][i]<-70):
                target.append(0)
                target_names.append('Bad')
            else:
                target.append(1)
                target_names.append('Interm.')

        df['combined_features']=combined_features
        colonne=['combined_features']
    elif kind=="pdr":                                            #It is for pdr
        for i in range(len(df)):
            if df['pdr'][i] < 0.3 :
                target.append(0)
                target_names.append('Bad')
            elif df['pdr'][i]>0.75 :
                target.append(2)
                target_names.append('Good')
            else:
                target.append(1)
                target_names.append('Interm.')
            colonne=['pdr']
    else:                                                          #It is for mean_rssi
        for i in range(len(df)):
            if df['mean_rssi'][i] < -70 :
                target.append(0)
                target_names.append('Bad')
            elif df['mean_rssi'][i]>-40 :
                target.append(2)
                target_names.append('Good')
            else:
                target.append(1)
                target_names.append('Interm.')
        colonne=['mean_rssi']
    df['target']=target
    df['target_names']=target_names
    colonne2=['target_names']
    df=df.loc[(df['src']==src)&(df['dst']==dst)]

    return df


#Logistic regression Classification
def my_logreg(df1,src,dst,size_of_test,kind,path):#Kind can be "pdr", "mean_rssi" or "all"
    df=my_split(df1,src,dst,size_of_test,kind)
    text=""
    if kind=="all":
        title=path+"LR\\Log_Reg_CM_ALL_"+str(src)+"==="+str(dst)+".png"
        title2=path+"LR\\Log_Reg_CM_Not_Normalized_ALL_"+str(src)+"==="+str(dst)+".png"
        subtitle="Accuracy = "
        colonne=['combined_features']
    elif kind=="pdr":
        title=path+"LR\\Log_Reg_CM_PDR_"+str(src)+"==="+str(dst)+".png"
        title2=path+"LR\\Log_Reg_CM_Not_Normalized_PDR_"+str(src)+"==="+str(dst)+".png"
        subtitle="Accuracy = "
        colonne=['pdr']
    else:
        title=path+"LR\\Log_Reg_CM_RSSI_"+str(src)+"==="+str(dst)+".png"
        title2=path+"LR\\Log_Reg_CM_Not_Normalized_RSSI_"+str(src)+"==="+str(dst)+".png"
        subtitle="Accuracy = "
        colonne=['mean_rssi']

    general_test_labels=list()
    general_pred_labels=list()
    channel_list=df1['channel'].unique()
    classes=df['target_names'].unique()
    accuracy_results=list()
    for i in range (len(channel_list)):
        channel_i=channel_list[i]
        colonne2=['target_names']
        ts=df.loc[(df['channel']==channel_i),colonne]
        ts2=df.loc[(df['channel']==channel_i),colonne2]

        features=ts.values
        labels=ts2['target_names']
        #classes=labels.unique()
        #if len(classes)>1:
         #   features, labels=rus.fit_resample(features, labels)#The ressampling strategy
         #   labels=pd.Series(labels)
        train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = size_of_test, random_state=None,shuffle=True)
        taill=len(pd.Series(train_labels).unique())
        if taill>1:
            LogReg=LogisticRegression()
            LogReg.fit(train_features,train_labels)
            pred_labels=LogReg.predict(test_features)
            general_test_labels=general_test_labels+list(test_labels)
            general_pred_labels=general_pred_labels+list(pred_labels)
        else:
            general_test_labels=general_test_labels+list(test_labels)
            general_pred_labels=general_pred_labels+list(test_labels)
    cm=confusion_matrix(general_test_labels,general_pred_labels)
    som=0
    taill=len(cm)
    total=0
    for i in range(taill):
        som=som+cm[i][i]
        for j in range(taill):
            total=total+cm[i][j]
        
    accuracy=som/total
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=True,title=subtitle)
    
    plt.savefig(title, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=False,title=subtitle)
    plt.savefig(title2, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    return accuracy
    
    
#SVM Classification
def my_svm(df1,src,dst,size_of_test,kind,path):#Kind can be "pdr", "mean_rssi" or "all"
    df=my_split(df1,src,dst,size_of_test,kind)
    if kind=="all":
            title=path+"SVM\\SVM_CM_ALL_"+str(src)+"==="+str(dst)+".png"
            title2=path+"SVM\\SVM_CM_Not_Normalized_ALL_"+str(src)+"==="+str(dst)+".png"
            subtitle="Accuracy = "
            colonne=['combined_features']
    elif kind=="pdr":
        title=path+"SVM\\SVM_CM_PDR_"+str(src)+"==="+str(dst)+".png"
        title2=path+"SVM\\SVM_CM_Not_Normalized_PDR_"+str(src)+"==="+str(dst)+".png"
        subtitle="Accuracy = "
        colonne=['pdr']
    else:
        title=path+"SVM\\SVM_CM_RSSI_"+str(src)+"==="+str(dst)+".png"
        title2=path+"SVM\\SVM_CM_Not_Normalized_RSSI_"+str(src)+"==="+str(dst)+".png"
        subtitle="Accuracy = "
        colonne=['mean_rssi']

    general_test_labels=list()
    general_pred_labels=list()
    channel_list=df1['channel'].unique()
    classes=df['target_names'].unique()
    accuracy_results=list()
    for i in range (len(channel_list)):
        channel_i=channel_list[i]
        colonne2=['target_names']
        ts=df.loc[(df['channel']==channel_i),colonne]
        ts2=df.loc[(df['channel']==channel_i),colonne2]

        features=ts.values
        labels=ts2['target_names']
        #classes=labels.unique()
        #if len(classes)>1:
        #    features, labels=rus.fit_resample(features, labels)#The ressampling strategy
        #    labels=pd.Series(labels)
        train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = size_of_test, random_state=None,shuffle=True)
        taill=len(pd.Series(train_labels).unique())
        if taill>1:
            clf = SVC(random_state=0, tol=1e-5)
            clf.fit(train_features, train_labels)
            pred_labels=clf.predict(test_features)
            general_test_labels=general_test_labels+list(test_labels)
            general_pred_labels=general_pred_labels+list(pred_labels)
        else:
            general_test_labels=general_test_labels+list(test_labels)
            general_pred_labels=general_pred_labels+list(test_labels)
    cm=confusion_matrix(general_test_labels,general_pred_labels)
    som=0
    taill=len(cm)
    total=0
    for i in range(taill):
        som=som+cm[i][i]
        for j in range(taill):
            total=total+cm[i][j]
        
    accuracy=som/total
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=True,title=subtitle)
    
    plt.savefig(title, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=False,title=subtitle)
    plt.savefig(title2, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    return accuracy
    
#Linear SVM Classification
def my_linear_svm(df1,src,dst,size_of_test,kind,path):#Kind can be "pdr", "mean_rssi" or "all"
    df=my_split(df1,src,dst,size_of_test,kind)
    if kind=="all":
        title=path+"LSVM\\LSVM_CM_ALL_"+str(src)+"==="+str(dst)+".png"
        title2=path+"LSVM\\LSVM_CM_Not_Normalized_ALL_"+str(src)+"==="+str(dst)+".png"
        subtitle="Accuracy = "
        colonne=['combined_features']
    elif kind=="pdr":
        title=path+"LSVM\\LSVM_CM_PDR_"+str(src)+"==="+str(dst)+".png"
        title2=path+"LSVM\\LSVM_CM_Not_Normalized_PDR_"+str(src)+"==="+str(dst)+".png"
        subtitle="Accuracy = "
        colonne=['pdr']
    else:
        title=path+"LSVM\\LSVM_CM_RSSI_"+str(src)+"==="+str(dst)+".png"
        title2=path+"LSVM\\LSVM_CM_Not_Normalized_RSSI_"+str(src)+"==="+str(dst)+".png"
        subtitle="Accuracy = "
        colonne=['mean_rssi']

    general_test_labels=list()
    general_pred_labels=list()
    channel_list=df1['channel'].unique()
    classes=df['target_names'].unique()
    accuracy_results=list()
    for i in range (len(channel_list)):
        channel_i=channel_list[i]
        colonne2=['target_names']
        ts=df.loc[(df['channel']==channel_i),colonne]
        ts2=df.loc[(df['channel']==channel_i),colonne2]

        features=ts.values
        labels=ts2['target_names']
        #classes=labels.unique()
        #if len(classes)>1:
        #    features, labels=rus.fit_resample(features, labels)#The ressampling strategy
         #   labels=pd.Series(labels)
        train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = size_of_test, random_state=None,shuffle=True)
        taill=len(pd.Series(train_labels).unique())
        if taill>1:
            clf = LinearSVC(random_state=0, tol=1e-5)
            clf.fit(train_features, train_labels)
            pred_labels=clf.predict(test_features)
            general_test_labels=general_test_labels+list(test_labels)
            general_pred_labels=general_pred_labels+list(pred_labels)
        else:
            general_test_labels=general_test_labels+list(test_labels)
            general_pred_labels=general_pred_labels+list(test_labels)
    cm=confusion_matrix(general_test_labels,general_pred_labels)
    som=0
    taill=len(cm)
    total=0
    for i in range(taill):
        som=som+cm[i][i]
        for j in range(taill):
            total=total+cm[i][j]
        
    accuracy=som/total
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=True,title=subtitle)
    
    plt.savefig(title, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=False,title=subtitle)
    plt.savefig(title2, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    return accuracy
    
#Random Forest Classification
def my_random_forest(df1,src,dst,size_of_test,kind,path):#Kind can be "pdr", "mean_rssi" or "all"
    df=my_split(df1,src,dst,size_of_test,kind)
    if kind=="all":
        title=path+"RF\\RF_CM_ALL_"+str(src)+"==="+str(dst)+".png"
        title2=path+"RF\\RF_CM_Not_Normalized_ALL_"+str(src)+"==="+str(dst)+".png"
        subtitle="Accuracy = "
        colonne=['combined_features']
    elif kind=="pdr":
        title=path+"RF\\RF_CM_PDR_"+str(src)+"==="+str(dst)+".png"
        title2=path+"RF\\RF_CM_Not_Normalized_PDR_"+str(src)+"==="+str(dst)+".png"
        subtitle="Accuracy = "
        colonne=['pdr']
    else:
        title=path+"RF\\RF_CM_RSSI_"+str(src)+"==="+str(dst)+".png"
        title2=path+"RF\\RF_CM_Not_Normalized_RSSI_"+str(src)+"==="+str(dst)+".png"
        subtitle="Accuracy = "
        colonne=['mean_rssi']

    general_test_labels=list()
    general_pred_labels=list()
    channel_list=df1['channel'].unique()
    classes=df['target_names'].unique()
    accuracy_results=list()
    for i in range (len(channel_list)):
        channel_i=channel_list[i]
        colonne2=['target_names']
        ts=df.loc[(df['channel']==channel_i),colonne]
        ts2=df.loc[(df['channel']==channel_i),colonne2]

        features=ts.values
        labels=ts2['target_names']
        #classes=labels.unique()
        #if len(classes)>1:
        #    features, labels=rus.fit_resample(features, labels)#The ressampling strategy
        #    labels=pd.Series(labels)
        train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = size_of_test, random_state=None,shuffle=True)
        taill=len(pd.Series(train_labels).unique())
        if taill>1:
            rf = RandomForestClassifier(n_estimators=1000, max_depth=5, random_state=0)
            rf.fit(train_features, train_labels)
            pred_labels=rf.predict(test_features)
            general_test_labels=general_test_labels+list(test_labels)
            general_pred_labels=general_pred_labels+list(pred_labels)
        else:
            general_test_labels=general_test_labels+list(test_labels)
            general_pred_labels=general_pred_labels+list(test_labels)
    cm=confusion_matrix(general_test_labels,general_pred_labels)
    som=0
    taill=len(cm)
    total=0
    for i in range(taill):
        som=som+cm[i][i]
        for j in range(taill):
            total=total+cm[i][j]
        
    accuracy=som/total
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=True,title=subtitle)
    
    plt.savefig(title, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=False,title=subtitle)
    plt.savefig(title2, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    return accuracy
    

def executor(df,src, dst, size_of_test,model_list,path):
    import os
    file_name=path+"final_results.csv"
    exists=os.path.isfile(file_name)
    if exists:
        file=open(file_name,"a+")
        #file.write(str('\n')+'model,link,test_size,accuracy_with_pdr_and_rssi,accuracy_with_pdr,accuracy_with_rssi')
    else:
        file=open(file_name,"a")
        file.write(str('\n')+'model,link,test_size,accuracy_with_pdr_and_rssi,accuracy_with_pdr,accuracy_with_rssi')
    for i in model_list:
        if i=='lr':
            lg_all=my_logreg(df,src,dst,size_of_test,'all',path)
            lg_pdr=my_logreg(df,src,dst,size_of_test,'pdr',path)
            lg_rssi=my_logreg(df,src,dst,size_of_test,'rssi',path)
            file.write(str('\n')+'LogReg,'+str(src)+'==='+str(dst)+','+str(size_of_test)+','+str(lg_all)+','+str(lg_pdr)+','+str(lg_rssi))
        elif i=='lsvm':
            lsvm_all=my_linear_svm(df,src,dst,size_of_test,'all',path)
            lsvm_pdr=my_linear_svm(df,src,dst,size_of_test,'pdr',path)
            lsvm_rssi=my_linear_svm(df,src,dst,size_of_test,'rssi',path)
            file.write(str('\n')+'LSVM,'+str(src)+'==='+str(dst)+','+str(size_of_test)+','+str(lsvm_all)+','+str(lsvm_pdr)+','+str(lsvm_rssi))
        elif i=='svm':
            svm_all=my_svm(df,src,dst,size_of_test,'all',path)
            svm_pdr=my_svm(df,src,dst,size_of_test,'pdr',path)
            svm_rssi=my_svm(df,src,dst,size_of_test,'rssi',path)
            file.write(str('\n')+'SVM,'+str(src)+'==='+str(dst)+','+str(size_of_test)+','+str(svm_all)+','+str(svm_pdr)+','+str(svm_rssi))
        elif i=='rf':  
            rf_all=my_random_forest(df,src,dst,size_of_test,'all',path)
            rf_pdr=my_random_forest(df,src,dst,size_of_test,'pdr',path)
            rf_rssi=my_random_forest(df,src,dst,size_of_test,'rssi',path)
            file.write(str('\n')+'RF,'+str(src)+'==='+str(dst)+','+str(size_of_test)+','+str(rf_all)+','+str(rf_pdr)+','+str(rf_rssi))
    file.close()
        
        
#The following function aims to compute the accuracies of all the links in the network for all our methods
def final_executor(df,size_of_test,my_execution_list,path):
    senders=df['src'].unique()
    receivers=df['dst'].unique()
    
    for sender in senders:
        for receiver in receivers:
            colonne=["pdr"]
            ts=df.loc[(df['src']==sender)&(df['dst']==receiver),colonne]
            if len(ts)>0:
                print("Starting link ",sender,"===",receiver)
                executor(df,sender,receiver,size_of_test,my_execution_list,path)
                print("Ending link ",sender,"===",receiver)

ros = RandomOverSampler(random_state=0)
rus = RandomUnderSampler(random_state=0)
path="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\"
my_execution_list=['lr', 'lsvm','svm','rf']
my_execution_list1=['lr']

executor(df,2, 47, 0.25,my_execution_list,path)
executor(df,3, 15, 0.25,my_execution_list,path)
#final_executor(df,0.25,my_execution_list,path)

C:\Users\WSN-LINK\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

C:\Users\WSN-LINK\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

C:\Users\WSN-LINK\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

C:\Users\WSN-LINK\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

C:\Users\WSN-LINK\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

C:\Users\WSN-LINK\Anaconda3\li

C:\Users\WSN-LINK\Anaconda3\lib\site-packages\matplotlib\pyplot.py:514: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).

C:\Users\WSN-LINK\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning:

The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.

C:\Users\WSN-LINK\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning:

The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.

C:\Users\WSN-LINK\Anaconda3\lib\site-packages\matplotlib\pyplot.py:514: RuntimeWarning:

More than 20 figures hav

<Figure size 216x216 with 0 Axes>

<Figure size 216x216 with 0 Axes>

<Figure size 216x216 with 0 Axes>

...

<Figure size 216x216 with 0 Axes>

<Figure size 216x216 with 0 Axes>

<Figure size 216x216 with 0 Axes>

<Figure size 216x216 with 0 Axes>

<Figure size 216x216 with 0 Axes>